In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from GenerationModule import generation
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader

excel_file = 'PID&SI.xlsx'
pd_data = pd.read_excel(excel_file, sheet_name='PDs')
suicide_data = pd.read_excel(excel_file, sheet_name='Suicide')

assert pd_data['serial'].equals(suicide_data['serial'])

X = pd_data.iloc[:, 1:26]
y = suicide_data['SI'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

class SuicideRiskMLP(nn.Module):
    def __init__(self):
        super(SuicideRiskMLP, self).__init__()
        self.fc1 = nn.Linear(25, 25)
        self.fc2 = nn.Linear(25, 14)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x) 
        return x

In [13]:
class SurrogateGeneration(torch.autograd.Function):
    @staticmethod
    def forward(ctx, params):

        with torch.no_grad():
            output = generation(params)
        ctx.save_for_backward(params)
        return output

    @staticmethod
    def backward(ctx, grad_output):

        params, = ctx.saved_tensors

        grad_params = grad_output * torch.ones_like(params)
        return grad_params


model2 = SuicideRiskMLP()
criterion = nn.BCELoss()
optimizer = optim.Adam(model2.parameters(), lr=0.01)
num_epochs = 100

for epoch in range(num_epochs):
    model2.train()
    optimizer.zero_grad()

    for i in range(5):
        input = X_train_tensor[i]
        label = y_train_tensor[i]

        params = model2(input)
        pred_result = SurrogateGeneration.apply(params)
        loss = criterion(pred_result, label)
        loss.backward()
        optimizer.step()

        print(loss)
        print(pred_result.grad)
        print(params.grad)

       
    break


tensor(100., grad_fn=<BinaryCrossEntropyBackward0>)
None
None
tensor(0., grad_fn=<BinaryCrossEntropyBackward0>)
None
None
tensor(100., grad_fn=<BinaryCrossEntropyBackward0>)
None
None
tensor(0., grad_fn=<BinaryCrossEntropyBackward0>)
None
None
tensor(100., grad_fn=<BinaryCrossEntropyBackward0>)
None
None


C:\Users\lilin\AppData\Local\Temp\ipykernel_816\3100050136.py:39: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten\src\ATen/core/TensorBody.h:494.)
  print(pred_result.grad)
C:\Users\lilin\AppData\Local\Temp\ipykernel_816\3100050136.py:40: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the lea